In [ ]:
!pip install natsort

In [ ]:
!pip install efficientnet_pytorch

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import natsort
import glob
import numpy as np
import pandas as pd
from torchvision import models
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from efficientnet_pytorch import EfficientNet

import torch.optim as optim
from torch.utils.data import Dataset,DataLoader

import torchvision
import torchvision.transforms as transforms

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(1218)
if device =='cuda':
    torch.cuda.manual_seed_all(1218)
print(device)

In [ ]:
efficient=EfficientNet.from_pretrained('efficientnet-b0')
efficient

In [ ]:
num_class = 7
num_ftrs = efficient._fc.in_features
efficient._fc = nn.Linear(num_ftrs, num_class)
efficient.to(device)
print(efficient)

In [ ]:
trans = transforms.Compose([
#     transforms.RandomRotation(30),
    transforms.CenterCrop(224),
    transforms.Grayscale(num_output_channels=3),
#     transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
#     transforms.Resize((224)),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

data = torchvision.datasets.ImageFolder('your_path',transform=trans)

In [ ]:
train_data,val_data=train_test_split(data, test_size=0.1)

In [ ]:
train_loader = DataLoader(dataset=train_data,batch_size=32,shuffle=True)
val_loader = DataLoader(dataset=val_data,batch_size=32,shuffle=True)

In [ ]:
optimizer = optim.SGD(efficient.parameters(), lr=0.003)
loss_func = torch.nn.CrossEntropyLoss().to(device)
lr_sche = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.001)

In [ ]:
epochs = 7

for epoch in range(epochs):  # loop over the dataset multiple times
    print("------- train -------")
    running_loss = 0.0
    lr_sche.step()
    for i, data in enumerate(train_loader):
        # get the inputs
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = efficient(inputs)
        loss = loss_func(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 32 == 31:    
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 32))
            
            running_loss = 0.0
    with torch.no_grad():
        efficient.eval()
        total=0
        correct=0
        for i, data in enumerate(val_loader):
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            out = efficient(inputs)
            predict = torch.argmax(out, dim=1)
            total += labels.size(0)
            correct += (predict==labels).sum()
    print("-- validation accuracy --")
    print("Accuracy : {:.2f}".format(100*correct/total))
    print()

        
print('Finish')

In [ ]:
torch.save(efficient.state_dict(), 'your_path')

In [ ]:
efficient = efficient.to(device)

In [ ]:
efficient.load_state_dict(torch.load('your_path'))

In [ ]:
class CustomDataSet(Dataset):

    def __init__(self, main_dir, transform):
        self.main_dir = main_dir
        self.transform = transform

        all_imgs = os.listdir(main_dir)
        self.total_imgs = natsort.natsorted(all_imgs)

    def __len__(self):
        return len(self.total_imgs)

    def __getitem__(self, idx):
        
        img_loc = os.path.join(self.main_dir, self.total_imgs[idx])
        image = Image.open(img_loc).convert("RGB")
        tensor_image = self.transform(image)
        

        return tensor_image

In [ ]:
test_data = CustomDataSet('your_path', transform=trans)

In [ ]:
test_set = DataLoader(dataset = test_data, batch_size = 32)

In [ ]:
result =[]
with torch.no_grad():
    file=pd.read_csv('your_path')
    
    for data in test_set:
        imgs = data
        imgs = imgs.to(device)
        prediction = efficient(imgs)
        result.append(torch.argmax(prediction,1).tolist())
        

    result_ = list(itertools.chain.from_iterable(result))
    prediction_np = np.array(result_)
    print(result_)

    file['value'] =result_
    file.to_csv('result.csv', index=False)       
